# Import libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import math
import random
import copy

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import Huber
from tensorflow.keras.models import load_model
from tensorflow.compat.v1.keras.layers import CuDNNLSTM

from collections import deque
from tqdm.notebook import tqdm
import pickle

*   Check GPU Available



In [ ]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

#DQN-RL

## Input state dataset

In [ ]:
# state = np.array([[1746, 2855,    0,    0,    0,    0,    0,    0,    0,    0,    0,
#            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
#            0,    0,    0,    0,    0,    0,    0,    0,    0, 3185,    0,
#            0,    0,    0,    0,    0,    0,    0,    0, 2445,    0,    0,
#            0,    0,    0,    0,    0,    0, 2095, 4087,    0,    0, 2137,
#         2298,    0, 4019, 3319, 2944, 2287, 2256, 3961],
#        [   0,    0, 2450, 2291, 1685, 1310,    0,    0,    0,    0,    0,
#            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
#            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
#            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
#            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
#            0,    0,    0,    0,    0,    0,    0,    0],
#        [   0,    0,    0,    0,    0,    0,    0,    0, 1197,    0,    0,
#            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
#            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
#            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
#            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
#            0,    0,    0,    0,    0,    0,    0,    0],
#        [   0,    0,    0,    0,    0,    0, 3106, 2371,    0, 3820, 3914,
#            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
#            0,    0,    0, 2638, 1316, 3201,    0,    0,    0,    0,    0,
#            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
#            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
#            0,    0,    0,    0,    0,    0,    0,    0],
#        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
#            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
#            0,    0,    0,    0,    0,    0, 3733,    0, 3915,    0, 2785,
#         2196,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
#         1989, 2314, 2113, 2288,    0,    0,    0,    0,    0,    0,    0,
#            0,    0,    0,    0,    0,    0,    0,    0],
#        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
#         1465, 1197, 2069, 3786, 3525, 1061, 2362, 2009, 3871, 4201, 1141,
#         3840,    0, 1426,    0,    0,    0,    0, 2290,    0,    0,    0,
#            0, 3649, 4097, 1732, 3967, 2636, 2902, 2742,    0, 1201, 2094,
#            0,    0,    0,    0, 1272, 3283,    0,    0, 1997, 2158,    0,
#            0,    0,    0,    0,    0,    0,    0,    0],
#        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
#            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
#            0, 2834,    0,    0,    0,    0,    0,    0,    0,    0,    0,
#            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
#            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
#            0,    0,    0,    0,    0,    0,    0,    0],
#        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
#            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
#            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
#            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
#            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
#            0, 3708,    0,    0,    0,    0,    0,    0]])

# state_init = copy.deepcopy(state)
# action_space = (state.shape[0]*state.shape[1]) - 1

# packet_amount = {'0': 1738, '1': 2754, '3': 2508, '4': 2343, '5': 1769, '7': 1409, '8': 3022, '9': 2369, '10': 1221, '12': 3664, '14': 3904, '15': 1535, '16': 1113, '17': 1953, '18': 3712, '19': 3618, '20': 1123, '21': 2436, '22': 2081, '23': 4026, '24': 4138, '25': 1166, '26': 3870, '27': 2816, '28': 1478, '29': 2545, '30': 1330, '31': 3224, '32': 3672, '33': 2142, '34': 3963, '36': 3240, '38': 2851, '39': 2232, '40': 3784, '41': 3958, '42': 1722, '43': 3973, '44': 2606, '45': 2919, '46': 2791, '47': 2500, '48': 1211, '50': 2098, '51': 1934, '52': 2304, '53': 2094, '55': 2361, '56': 1233, '57': 3307, '58': 2010, '59': 4108, '60': 1957, '61': 2132, '63': 2177, '64': 2407, '66': 3672, '67': 4148, '68': 3453, '69': 2941, '70': 2311, '71': 2285, '72': 3891}
# controller_packet_amount_copy = {'2': 39963, '6': 8029, '11': 1221, '13': 20058, '35': 21411, '49': 68082, '54': 2816, '65': 3672}
# capacity = 30000

In [ ]:
state = np.array([[   0,    0,    0,    0,    0, 1699],
                  [2326, 2568, 2103, 3457, 1976,    0],
                  [   0,    0,    0,    0,    0,    0]])

# state = state.astype('float32')
state_init = copy.deepcopy(state)
packet_amount = {'2': 2326, '4': 2568, '5': 2103, '6': 3457, '7': 1976, '9': 1699}
capacity = 6_000
action_space = (state.shape[0]*state.shape[1]) - 1

In [ ]:
sA = sparse.csr_matrix(state)

## Build model

In [ ]:
def build_neural_network(state):
    model = Sequential()

    # Given the state shape, consider rows as timesteps and columns as features
    timesteps = state.shape[0]
    features = state.shape[1]

    # LSTM layer
    # model.add(LSTM(128, input_shape=(timesteps, features), return_sequences=True))
    # model.add(LSTM(128))
    model.add(CuDNNLSTM(64, input_shape=(timesteps, features), return_sequences=True))
    model.add(CuDNNLSTM(64))

    # Dense layers
    model.add(Dense(512, activation='relu'))
    model.add(Dense(timesteps * features + 1))

    model.compile(optimizer=RMSprop(), loss=Huber())
    return model

online_network = build_neural_network(state)
target_network = build_neural_network(state)

In [ ]:
online_network.summary()



*   Parameters of model training



In [ ]:
memory_size = 10_000
reward_discount_factor = 0.2
number_of_ep = 200
epsilon = 1.0
epsilon_decay_factor = 0.999
epsilon_min = 0.01
batch_size = 32
target_network_update_int = 100

##Environtment setup instruction

In [ ]:
def migratefromAction(action, state, packet_amount):
  state = np.copy(state)
  controllers = state.shape[0]  #Rows
  switches = state.shape[1]     #Column
  packet_amount_np = np.array(list(packet_amount.values()))
  action = action + 1
  target_con = (math.ceil(action/switches))
  target_sw = (action%switches)

  if (action%switches) == 0:
    target_sw = switches

  target_con = target_con -1
  target_sw = target_sw - 1

  # mapping_copy = copy.deepcopy(mapping)
  state[state > 0] = 1
  state[target_con][target_sw] = 1
  for i in range(controllers):
    if i != target_con:
      state[i][target_sw] = 0
  state = np.multiply(state, packet_amount_np)
  return state

# res = migratefromAction(12, state1, packet_amount)
# print(state1)
# print(res)

In [ ]:
def calculateNet(state, lambda_, etha):
  # Calculate CV based on Equation 5
  controller_loads = np.sum(state, axis=1)
  mu = np.mean(controller_loads)
  sigma = np.std(controller_loads)
  CV = sigma / mu
  net_value = math.exp(lambda_ * etha - (1 - lambda_) * CV)

  return net_value

In [ ]:
def calculateReward(state, next_state):
  etha = 0.3
  lambda_ = 0.7

  net_lastcon = calculateNet(state, lambda_, etha)
  net_nowcon = calculateNet(next_state, lambda_, etha)
  delta_net = net_nowcon - net_lastcon
  if delta_net > 0:
    r = 1
  elif delta_net == 0:
    r = -0.1
  else:
    r = -10

  return r

In [ ]:
def checkDone(next_state, capacity):
  row_sums = np.sum(next_state, axis=1)
  done = not np.any(row_sums > capacity)
  return done

In [ ]:
def step(action, state, packet_amount, capacity):

  next_state = migratefromAction(action, state, packet_amount)
  reward = calculateReward(state, next_state)
  done = checkDone(next_state, capacity)

  return next_state, reward, done

## Training

In [ ]:
## สร้าง List เอาไว้เก็บประวัติ reward รวมของแต่ละ ep เพื่อดูการพัฒนา
reward_hist = []
t = 0
## สร้าง memory หรือ D
memory = deque(maxlen=memory_size)
for ep in tqdm(range(number_of_ep+1)):
  #Initial state
  state = state_init
  reward_of_ep = 0
  while True:
    ## เลือก action
    optimal_action = np.argmax(online_network.predict(state.reshape(1, state.shape[0], state.shape[1], 1))[0])
    random_action = random.randint(0,action_space)
    action = np.random.choice([random_action,optimal_action],p=[epsilon,1-epsilon])
    ## ทำ action ใส่ environment
    next_state,reward,done = step(action, state, packet_amount, capacity)
    ## เก็บ experience ลงใน memory
    memory.append([state,action,reward,next_state,done])
    ## รอให้เก็บ experience ให้ได้ตาม batch size ก่อน
    if len(memory)>=batch_size:
      ## Sample experience จาก memory
      batch = np.array(random.sample(memory, batch_size))
      ## หาค่า target
      target = batch[:,2] + reward_discount_factor*(np.max(target_network.predict(np.stack(batch[:,3])),axis=1)) * (1-batch[:,4])
      ## หาค่าพรีดิกในปัจจุบัน
      current_Q = target_network.predict(np.stack(batch[:,0]))
      ## สร้าง target value list
      current_Q[np.arange(batch_size),list(batch[:,1])] = target
      ## Fit model
      online_network.fit(np.stack(batch[:,0]), current_Q, verbose=False)
    ## ก้าวไป next state
    state = next_state
    ## ลดค่า epsilon
    if epsilon>epsilon_min: epsilon = epsilon*epsilon_decay_factor
    ## รวมค่า reward_of_ep (เอาไว้ดูเฉยๆ)
    reward_of_ep += reward
    ## อัพเดท target network
    if t%target_network_update_int==0:
      target_network.set_weights(online_network.get_weights())
    if done:
      break
    t += 1
  reward_hist.append(reward_of_ep)
  ## Save model เอาไว้วัดผลในภายหลัง
  if ep%20==0:
    online_network.save('/content/dqn/model/{}.h5'.format(ep))
    pickle.dump(reward_hist,open('/content/dqn/score.pkl','wb'))

##Testing

In [ ]:
online_network = load_model('/content/dqn/model/{}.h5'.format(100))

In [ ]:
state = state_init
print(state, packet_amount, capacity)



*   Action predicition



In [ ]:
reward_of_ep = 0
list_action = []
state = state_init
while True:
  ## เลือก action
  optimal_action = np.argmax(online_network.predict(state.reshape(1, state.shape[0], state.shape[1], 1))[0])
  ## ทำ action ใส่ environment
  next_state,reward,done = step(optimal_action, state, packet_amount, capacity)
  ## รวมค่า reward_of_ep (เอาไว้ดูเฉยๆ)
  reward_of_ep += reward
  ## ก้าวไป next state
  state = next_state
  list_action.append(optimal_action)
  if done:
    break



*   List of predicted action



In [ ]:
print(list_action)
print(state)



*   Reward history



In [ ]:
reward_hist = pickle.load(open('/content/dqn/score.pkl','rb'))
plt.figure(figsize=(10,5))
plt.plot(reward_hist)
plt.xlabel('Episode')
plt.ylabel('Reward')